# Backtest Strategy
* Use Backtrader for event-driven backtest

In [1]:
import pandas as pd
import numpy as np
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
import seaborn as sns

from feature_engineering import Indicators
from portfolio_optimizer import portfolio_optimizer
from data_sampler import data_manager
from random_forest import random_forest
from test_indicators import TestIndicators
import bt

### Read Data

In [ ]:
# read data from local file
data = data_manager().readPair(pairs = "BTCUSD ETHUSD BATUSD FILUSD MKRUSD UNIUSD ZRXUSD",
                               start='2020-11-01', end='2021-12-31')
data.head()

In [ ]:
# Cusum filter to sample data points
# threshold
threshold = 0.02
closes = data.reset_index(level=0).pivot(columns='level_0', values='close')

# sampling process
samples = data_manager().cusumFilter(closes, h = threshold)
samples.keys()

In [ ]:
pd.DataFrame([len(samples[i]) for i in samples], index = samples.keys(), columns=['count'])

### Generate Features
* Prediction target is the return of the next hourly bar

In [ ]:
# set parameters
params = {'ADX': {'window': 10}, 'CCI': {'window': 10}, 'MACD': {'fast': 12, 'slow': 26, 'signal': 9}, 
          'MOM': {'window': 5}, 'RSI': {'window': 10}, 'WILLR': {'window': 10}, 'OBV': {}, 
          'Regression': {'window': 10}, 'Volatility': {'window': 5, 'nbdev': 1}}

TA = {}

for ticker in data.index.get_level_values(0).unique():
    # get features
    TA[ticker] = Indicators().pool(data.loc[ticker], params=params)
    TA[ticker] = TA[ticker].loc[samples[ticker]]
    
    # get targets
    TA[ticker]['Return'] = data.close.loc[ticker].pct_change().shift(-1).loc[samples[ticker]]

TA = pd.concat(TA, axis = 0)
TA.tail()

In [15]:
plt.figure(figsize = (20, 10))
TA.loc['ZRXUSD'].plot()
plt.show()

In [ ]:
s = bt.Strategy('pair trading', [bt.algos.RunEveryNPeriods(1),
                       bt.algos.SelectAll(),
                       bt.algos.WeighTarget(signal),
                       bt.algos.Rebalance()])

test = bt.Backtest(s, df.loc[signal.index, ['zec/eth/btc', 'zec/btc']])
res = bt.run(test)

In [ ]:
res.display()